# GTSRB & RGB

In [1]:
import numpy as np
import warnings
from utils import *
from calibrators import * 
import pickle
warnings.filterwarnings("ignore")
import torch
import sys
sys.path.append('../')
import time
import copy
from SLURM.sklearn_config import *
import time
import tensorflow as tf
from skimage import color
from skimage import io
from keras.datasets import cifar10



In [18]:
dataset_name, model_name, shuffle='CIFAR_RGB','RF',0
model_info= load_model(dataset_name, model_name, shuffle)
(train_X, train_y), (test_X, test_y) = cifar10.load_data()
img=train_X[0:10]
imgGray = color.rgb2gray(img)
print(img.shape,imgGray.shape)

(10, 32, 32, 3) (10, 32, 32)


In [10]:
img

array([[[ 59,  62,  63],
        [ 43,  46,  45],
        [ 50,  48,  43],
        ...,
        [158, 132, 108],
        [152, 125, 102],
        [148, 124, 103]],

       [[ 16,  20,  20],
        [  0,   0,   0],
        [ 18,   8,   0],
        ...,
        [123,  88,  55],
        [119,  83,  50],
        [122,  87,  57]],

       [[ 25,  24,  21],
        [ 16,   7,   0],
        [ 49,  27,   8],
        ...,
        [118,  84,  50],
        [120,  84,  50],
        [109,  73,  42]],

       ...,

       [[208, 170,  96],
        [201, 153,  34],
        [198, 161,  26],
        ...,
        [160, 133,  70],
        [ 56,  31,   7],
        [ 53,  34,  20]],

       [[180, 139,  96],
        [173, 123,  42],
        [186, 144,  30],
        ...,
        [184, 148,  94],
        [ 97,  62,  34],
        [ 83,  53,  34]],

       [[177, 144, 116],
        [168, 129,  94],
        [179, 142,  87],
        ...,
        [216, 184, 140],
        [151, 118,  84],
        [123,  92,  72]]

In [16]:
(img.reshape(1, -1).astype(np.float64).reshape(1,32,32,3)==img).all()

True

In [20]:
#avg(2)
all_time=[]
for dataset_name in ['CIFAR_RGB']:#'GTSRB_RGB',
    for model_name in ['RF','GB','pytorch']:
        for shuffle in range(10):
            polling = torch.nn.AvgPool2d(2)
            model_info= load_model(dataset_name, model_name, shuffle)
            data=model_info.data
            pixels=int(sqrt(data.X_train.shape[1]/3))
            train=model_info.data.X_train.reshape(len(data.X_train),32,32,3)
            imgGray_train = color.rgb2gray(train)
            pool_train=polling(torch.tensor(imgGray_train)).reshape(len(data.X_train),-1)
            
            test=model_info.data.X_test.reshape(len(data.X_test),32,32,3)
            imgGray_test = color.rgb2gray(test)
            pool_test=polling(torch.tensor(imgGray_test)).reshape(len(data.X_test),-1)
                               
            val=model_info.data.X_val.reshape(len(data.X_val),32,32,3)
            imgGray_val = color.rgb2gray(val)
            pool_val=polling(torch.tensor(imgGray_val)).reshape(len(data.X_val),-1)
                               
                               
            num_label=model_info.data.num_labels
            stability,time_all,ex_in_time=new_stability_calc(np.array(pool_train),np.array(pool_val),model_info.data.y_train,model_info.y_pred_val,num_label)
            stability_test,_,_=new_stability_calc(np.array(pool_train),np.array(pool_test),model_info.data.y_train,model_info.y_pred_test,num_label)

            all_time.append((model_name,time_all,ex_in_time))
            np.save(f'./stab/{dataset_name}/gray_stab_2pool_{dataset_name}_{model_name}_{shuffle}.npy',stability)
            np.save(f'./stab/{dataset_name}/gray_test_stab_2pool_{dataset_name}_{model_name}_{shuffle}.npy',stability_test)

all_time

KeyboardInterrupt: 

In [ ]:
#avg(2)
all_time=[]
for dataset_name in ['CIFAR_RGB']:#'GTSRB_RGB',
    for model_name in ['pytorch']:
        for shuffle in range(10):
            polling = torch.nn.AvgPool2d(2)
            if model_name=='pytorch':
                model_info=load_model_pytorch(dataset_name, model_name, shuffle)
            else:
                model_info= load_model(dataset_name, model_name, shuffle)
            data=model_info.data
            pixels=int(sqrt(data.X_train.shape[1]/3))
            model_info.data.X_train=polling(torch.tensor(model_info.data.X_train.reshape((len(data.X_train),3,pixels,pixels)))).reshape(len(data.X_train),-1)
            model_info.data.X_test=polling(torch.tensor(model_info.data.X_test.reshape((len(data.X_test),3,pixels,pixels)))).reshape(len(data.X_test),-1)
            model_info.data.X_val=polling(torch.tensor(model_info.data.X_val.reshape((len(data.X_val),3,pixels,pixels)))).reshape(len(data.X_val),-1)
            num_label=model_info.data.num_labels
            stability,time_all,ex_in_time=new_stability_calc(np.array(model_info.data.X_train),np.array(model_info.data.X_val),model_info.data.y_train,model_info.y_pred_val,num_label)
            stability_test,_,_=new_stability_calc(np.array(model_info.data.X_train),np.array(model_info.data.X_test),model_info.data.y_train,model_info.y_pred_test,num_label)

            all_time.append((model_name,time_all,ex_in_time))
            np.save(f'./stab/{dataset_name}/stab_2pool_{dataset_name}_{model_name}_{shuffle}.npy',stability)
            np.save(f'./stab/{dataset_name}/test_stab_2pool_{dataset_name}_{model_name}_{shuffle}.npy',stability_test)

all_time

In [ ]:
#avg(3)
all_time=[]
for dataset_name in ['CIFAR_RGB','GTSRB_RGB']:
    for model_name in ['RF','GB']:#,'pytorch']:
        for shuffle in range(10):
            polling = torch.nn.AvgPool2d(3)
            if model_name=='pytorch':
                model_info=load_model_pytorch(dataset_name, model_name, shuffle)
            else:
                model_info= load_model(dataset_name, model_name, shuffle)
            data=model_info.data
            pixels=int(sqrt(data.X_train.shape[1]/3))
            model_info.data.X_train=polling(torch.tensor(model_info.data.X_train.reshape((len(data.X_train),3,pixels,pixels)))).reshape(len(data.X_train),-1)
            model_info.data.X_test=polling(torch.tensor(model_info.data.X_test.reshape((len(data.X_test),3,pixels,pixels)))).reshape(len(data.X_test),-1)
            model_info.data.X_val=polling(torch.tensor(model_info.data.X_val.reshape((len(data.X_val),3,pixels,pixels)))).reshape(len(data.X_val),-1)
            num_label=model_info.data.num_labels
            stability,time_all,ex_in_time=new_stability_calc(np.array(model_info.data.X_train),np.array(model_info.data.X_val),model_info.data.y_train,model_info.y_pred_val,num_label)
            stability_test,_,_=new_stability_calc(np.array(model_info.data.X_train),np.array(model_info.data.X_test),model_info.data.y_train,model_info.y_pred_test,num_label)

            all_time.append((model_name,time_all,ex_in_time))
            np.save(f'./stab/{dataset_name}/stab_3pool_{dataset_name}_{model_name}_{shuffle}.npy',stability)
            np.save(f'./stab/{dataset_name}/test_stab_3pool_{dataset_name}_{model_name}_{shuffle}.npy',stability_test)

all_time

In [ ]:
#max(2)
all_time=[]
for dataset_name in ['CIFAR_RGB','GTSRB_RGB']:
    for model_name in ['RF','GB']:#,'pytorch']:
        for shuffle in range(10):
            polling = torch.nn.MaxPool2d(2)
            if model_name=='pytorch':
                model_info=load_model_pytorch(dataset_name, model_name, shuffle)
            else:
                model_info= load_model(dataset_name, model_name, shuffle)
            data=model_info.data
            pixels=int(sqrt(data.X_train.shape[1]/3))
            model_info.data.X_train=polling(torch.tensor(model_info.data.X_train.reshape((len(data.X_train),3,pixels,pixels)))).reshape(len(data.X_train),-1)
            model_info.data.X_test=polling(torch.tensor(model_info.data.X_test.reshape((len(data.X_test),3,pixels,pixels)))).reshape(len(data.X_test),-1)
            model_info.data.X_val=polling(torch.tensor(model_info.data.X_val.reshape((len(data.X_val),3,pixels,pixels)))).reshape(len(data.X_val),-1)
            num_label=model_info.data.num_labels
            stability,time_all,ex_in_time=new_stability_calc(np.array(model_info.data.X_train),np.array(model_info.data.X_val),model_info.data.y_train,model_info.y_pred_val,num_label)
            stability_test,_,_=new_stability_calc(np.array(model_info.data.X_train),np.array(model_info.data.X_test),model_info.data.y_train,model_info.y_pred_test,num_label)

            all_time.append((model_name,time_all,ex_in_time))
            np.save(f'./stab/{dataset_name}/stab_2maxpool_{dataset_name}_{model_name}_{shuffle}.npy',stability)
            np.save(f'./stab/{dataset_name}/test_stab_2maxpool_{dataset_name}_{model_name}_{shuffle}.npy',stability_test)

all_time

# convert RGB to grayscale

In [ ]:
#avg(2)
all_time=[]
for dataset_name in ['CIFAR_RGB','GTSRB_RGB']:
    for model_name in ['RF','GB']:#,'pytorch']:
        for shuffle in range(10):
            polling = torch.nn.AvgPool2d(2)
            if model_name=='pytorch':
                model_info=load_model_pytorch(dataset_name, model_name, shuffle)
            else:
                model_info= load_model(dataset_name, model_name, shuffle)
            data=model_info.data

            model_info.data.X_train=tf.image.rgb_to_grayscale(torch.tensor(model_info.data.X_train))
            model_info.data.X_test=tf.image.rgb_to_grayscale(torch.tensor(model_info.data.X_test))
            model_info.data.X_val=tf.image.rgb_to_grayscale(torch.tensor(model_info))
            
            pixels=int(sqrt(data.X_train.shape[1]))
            model_info.data.X_train=polling(model_info.data.X_train.reshape((len(data.X_train),pixels,pixels))).reshape(len(data.X_train),-1)
            model_info.data.X_test=polling(model_info.data.X_test.reshape((len(data.X_test),pixels,pixels))).reshape(len(data.X_test),-1)
            model_info.data.X_val=polling(model_info.data.X_val.reshape((len(data.X_val),pixels,pixels))).reshape(len(data.X_val),-1)
            num_label=model_info.data.num_labels
            stability,time_all,ex_in_time=new_stability_calc(np.array(model_info.data.X_train),np.array(model_info.data.X_val),model_info.data.y_train,model_info.y_pred_val,num_label)
            stability_test,_,_=new_stability_calc(np.array(model_info.data.X_train),np.array(model_info.data.X_test),model_info.data.y_train,model_info.y_pred_test,num_label)

            all_time.append((model_name,time_all,ex_in_time))
           
            np.save(f'./stab/{dataset_name}/stab_2pool_{dataset_name}_gray_{model_name}_{shuffle}.npy',stability)
            np.save(f'./stab/{dataset_name}/test_stab_2pool_{dataset_name}_gray_{model_name}_{shuffle}.npy',stability_test)

all_time



In [3]:
def new_stability_calc(trainX,testX,train_y,test_y_pred,num_labels):
    '''
    Calculates the stability of the test set.
            Parameters:
                    trainX (List)
                    testX (List) 
                    train_y (List)
                    test_y_pred (list)
                    num_labels (Int)
            Returns:
                    stability(List)
    '''    
    time_lst = []
    same_nbrs=[]
    other_nbrs=[]
    for i in range(num_labels):
        idx_other=np.where(train_y!=i)
        other_nbrs.append(NearestNeighbors(n_neighbors=1).fit(trainX[idx_other]))
        idx_same=np.where(train_y==i)
        same_nbrs.append(NearestNeighbors(n_neighbors=1).fit(trainX[idx_same]))
    
    
    stability=np.array([-1.]*testX.shape[0])
    start = time.time()
    for i in range(testX.shape[0]):
        x=testX[i]
        pred_label=test_y_pred[i]
        dist1,idx1= same_nbrs[pred_label].kneighbors([x])
        dist2,idx2= other_nbrs[pred_label].kneighbors([x])

        stability[i]=(dist2-dist1)/2
    end = time.time()
    time_all=end-start
    ex_in_time=testX.shape[0]/time_all
    return stability,time_all,ex_in_time

In [ ]:
#max(2)
all_time=[]
for dataset_name in ['GTSRB_RGB']:#,'GTSRB_RGB']:
    for model_name in ['GB','RF','pytorch']:
        for shuffle in range(10):
            polling = torch.nn.MaxPool2d(2)
            if model_name=='pytorch':
                model_info=load_model_pytorch(dataset_name, model_name, shuffle)
            else:
                model_info= load_model(dataset_name, model_name, shuffle)
            data=model_info.data
            pixels=int(sqrt(data.X_train.shape[1]/3))
            model_info.data.X_train=polling(torch.tensor(model_info.data.X_train.reshape((len(data.X_train),3,pixels,pixels)))).reshape(len(data.X_train),-1)
            model_info.data.X_test=polling(torch.tensor(model_info.data.X_test.reshape((len(data.X_test),3,pixels,pixels)))).reshape(len(data.X_test),-1)
            model_info.data.X_val=polling(torch.tensor(model_info.data.X_val.reshape((len(data.X_val),3,pixels,pixels)))).reshape(len(data.X_val),-1)
            num_label=model_info.data.num_labels
            stability,time_all,ex_in_time=new_stability_calc(np.array(model_info.data.X_train),np.array(model_info.data.X_val),model_info.data.y_train,model_info.y_pred_val,num_label)
            stability_test,_,_=new_stability_calc(np.array(model_info.data.X_train),np.array(model_info.data.X_test),model_info.data.y_train,model_info.y_pred_test,num_label)

            all_time.append((model_name,time_all,ex_in_time))
            np.save(f'./stab/{dataset_name}/stab_2maxpool_{dataset_name}_{model_name}_{shuffle}.npy',stability)
            np.save(f'./stab/{dataset_name}/test_stab_2maxpool_{dataset_name}_{model_name}_{shuffle}.npy',stability_test)

all_time

In [5]:
#avg(4)
all_time=[]
for dataset_name in ['SignLanguage']:
    for model_name in ['RF']:
        shuffle=0
        model_info= load_model(dataset_name, model_name, shuffle)
        data=model_info.data
        num_label=model_info.data.num_labels
        stability,time_all,ex_in_time=stability_calc_pool(np.array(model_info.data.X_train),np.array(model_info.data.X_val),model_info.data.y_train,model_info.y_pred_val,num_label,pool_type='Avg',pool_size=4)
        stability_test,_,_=stability_calc_pool(np.array(model_info.data.X_train),np.array(model_info.data.X_test),model_info.data.y_train,model_info.y_pred_test,num_label,pool_type='Avg',pool_size=4)

        all_time.append((model_name,time_all,ex_in_time))
print("avg4",all_time)


ValueError: only one element tensors can be converted to Python scalars

In [ ]:

def stability_calc_pool(trainX,testX,train_y,test_y_pred,num_labels,pool_type='Avg',pool_size=1):
    '''
    Calculates the stability of the test set.
            Parameters:
                    trainX (List)
                    testX (List) 
                    train_y (List)
                    test_y_pred (list)
                    num_labels (Int)
            Returns:
                    stability(List)
    '''    
    time_lst = []
    same_nbrs=[]
    other_nbrs=[]
    for i in range(num_labels):
        idx_other=np.where(train_y!=i)
        other_nbrs.append(NearestNeighbors(n_neighbors=1).fit(trainX[idx_other]))
        idx_same=np.where(train_y==i)
        same_nbrs.append(NearestNeighbors(n_neighbors=1).fit(trainX[idx_same]))
    
    if pool_type=='Avg':
        polling = torch.nn.AvgPool2d(pool_size)
    elif pool_type=='Max':
        polling = torch.nn.MaxPool2d(pool_size)
    pixels=int(sqrt(trainX.shape[1]))
    stability=np.array([-1.]*testX.shape[0])
    start = time.time()
    for i in range(testX.shape[0]):
        x=polling(torch.tensor(testX[i].reshape((1,pixels,pixels))))
        pred_label=test_y_pred[i]
        
        dist1,idx1= same_nbrs[pred_label].kneighbors([x])
        dist2,idx2= other_nbrs[pred_label].kneighbors([x])

        stability[i]=(dist2-dist1)/2
    end = time.time()
    time_all=end-start
    ex_in_time=testX.shape[0]/time_all
    return stability,time_all,ex_in_time